In [1]:
import torch
import torch.nn as nn
from Model import model, modelArguments
import time
import re
import math
import triton

In [2]:
fmodel = model(modelArguments)

state_dict = torch.load('./checkpoints/model_weightsBPE6.pth')
new_state_dict = {k.replace('_orig_mod.', ''): v for k, v in state_dict.items()}
fmodel.load_state_dict(new_state_dict, strict=False)

fmodel = fmodel.to('cuda')

/tmp/ipykernel_44869/3021890348.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('./checkpoints/model_weightsBPE6.pth')


In [3]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="large_tokenizer.json")


def tokenize(text, second = False):
    ids = tokenizer.encode(text, add_special_tokens=True)
    ids1 = [1] + ids
    ids1 = torch.tensor(ids1, dtype=torch.int16)
    
    # # Pad or truncate the sequence to tholtae max_seq_length
    # if ids1.size(0) < modelArguments.max_seq_len:
    #     padding_length = modelArguments.max_seq_len - ids1.size(0)
    #     ids1 = torch.cat([ids1, torch.zeros(padding_length, dtype=torch.int16)])
        
    # elif ids1.size(0) >= modelArguments.max_seq_len:
    #     ids1 = ids1[:modelArguments.max_seq_len]
    #     ids1[-1] = 2
    return ids1

/home/frosthead/.conda/envs/DL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def id_to_token(x):
    s = ''
    for i in x[0]:
        # j = torch.argmax(i)
        s += tokenizer.decode(i)

    return s

def id_to_token_M(x):
    s = ''
    for i in x[0]:
        j = torch.argmax(i)
        s += tokenizer.decode(j)

    return s

In [5]:
fmodel.eval()

model(
  (shared_emb): shared_emb()
  (enc_layers): ModuleList(
    (0-9): 10 x EncoderBlock(
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attention): FlashAttention(
        (q_linear): Linear(in_features=512, out_features=512, bias=False)
        (k_linear): Linear(in_features=512, out_features=512, bias=False)
        (v_linear): Linear(in_features=512, out_features=512, bias=False)
        (out_linear): Linear(in_features=512, out_features=512, bias=False)
      )
      (feedforward): FeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=False)
        (fc2): Linear(in_features=2048, out_features=512, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
)

In [10]:
# test sentence

sentence = "the the same,, a aated, the world,, a a,,,, you are a to a,"
generated_ids  = tokenize(sentence).to(torch.long).to('cuda').unsqueeze(0)

print(generated_ids)

tensor([[    1,  1313,   265,  1300, 20170,   261,   261,   663,    15,   265,
          1602, 20170,   261,   261, 20170, 20170,   377,   418,   261,   299,
           261,    15]], device='cuda:0')


In [11]:


for _ in range(20):  # Generate up to 20 tokens
    with torch.no_grad():
        logits = fmodel(generated_ids)  # Pass the current sequence to the model
        next_token_logits = logits[:, -1, :]  # Get logits for the last token

    # Predict the next token (greedy decoding)
    next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)  # Shape: [1, 1]

    # Append the predicted token to the sequence
    generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

    # Stop if EOS token is predicted
    if next_token_id.item() == 2:
        print("eos")
        break

# Decode the generated sequence
output_text = tokenizer.decode(generated_ids.squeeze())
print("Generated text:", output_text)


Generated text: [SOS]the the same,, a aated, the world,, a a,,,, you are a to a, you can be a own of the best of the best of the best of the best of the best
